In [1]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, KNNWithMeans
from surprise import accuracy
from surprise.model_selection import train_test_split
import pickle

# Đọc dữ liệu từ file ratings.csv
ratings = pd.read_csv('ratings.csv')

# Chuyển cột CustomerID thành chuỗi để tránh lỗi khi xử lý
ratings['CustomerID'] = ratings['CustomerID'].astype(str)

# Chuẩn hóa dữ liệu với Surprise Reader
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(ratings[['CustomerID', 'stockCodeTransform', 'rate']], reader)

# Lớp User-Based Collaborative Filtering
class UserBasedWithTopItems(KNNWithMeans):
    def __init__(self, sim_options={'name': 'cosine', 'user_based': True}, *args, **kwargs):
        super().__init__(sim_options=sim_options, *args, **kwargs)

    def get_anti_test_set_for_user(self, user_id):
        fill = self.trainset.global_mean
        anti_testset = []
        u = self.trainset.to_inner_uid(user_id)
        user_items = set([j for (j, _) in self.trainset.ur[u]])
        anti_testset += [(self.trainset.to_raw_uid(u), self.trainset.to_raw_iid(i), fill) 
                         for i in self.trainset.all_items() if i not in user_items]
        return anti_testset

    def get_top_items(self, user_id, k=10):
        testSet = self.get_anti_test_set_for_user(user_id)
        predictions = self.test(testSet)

        recommendations = []
        for _, itemID, _, estimatedRating, _ in predictions:
            recommendations.append((itemID, estimatedRating))
        recommendations.sort(key=lambda x: x[1], reverse=True)
        return recommendations[:k]

# Chia dữ liệu thành tập huấn luyện và kiểm tra (80% train, 20% test)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Tạo đối tượng từ lớp UserBasedWithTopItems
model = UserBasedWithTopItems()

# Huấn luyện mô hình trên tập huấn luyện
model.fit(trainset)

# Dự đoán trên tập huấn luyện
train_predictions = model.test(trainset.build_testset())

# Tính RMSE trên tập huấn luyện
rmse_train = accuracy.rmse(train_predictions)

# Dự đoán trên tập kiểm tra
predictions = model.test(testset)

# Tính RMSE trên tập kiểm tra
rmse_test = accuracy.rmse(predictions)


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8714
RMSE: 0.9608


In [2]:

# Lưu mô hình vào tệp
with open('item_based_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Tải mô hình đã lưu
with open('item_based_model.pkl', 'rb') as file:
    model = pickle.load(file)


In [3]:
# Tạo tập dữ liệu đầy đủ 
full_data = data.build_full_trainset()

In [4]:


# Tạo danh sách người dùng và sản phẩm
user_ids = [full_data.to_raw_uid(u) for u in range(full_data.n_users)]  # Lấy danh sách tất cả người dùng
item_ids = [full_data.to_raw_iid(i) for i in range(full_data.n_items)]  # Lấy danh sách tất cả sản phẩm

# Tạo ma trận Yhat với đúng thứ tự: hàng là sản phẩm, cột là người dùng
Yhat = np.zeros((len(item_ids), len(user_ids)))  # Khởi tạo ma trận dự đoán

# Điền giá trị dự đoán vào ma trận
for item_idx, item_id in enumerate(item_ids):  # Sản phẩm làm hàng
    for user_idx, user_id in enumerate(user_ids):  # Người dùng làm cột
        est = model.predict(user_id, item_id).est
        Yhat[item_idx, user_idx] = est

# Làm tròn giá trị đến chữ số thập phân thứ nhất
Yhat = np.round(Yhat, 1)

In [5]:
# Lưu ma trận Yhat vào file numpy
np.save('Yhat_UB.npy', Yhat)

# Để tải lại ma trận Yhat sau này
Yhat_loaded = np.load('Yhat_UB.npy')
print(Yhat_loaded)
Yhat_loaded.shape[0]

[[4.7 4.9 4.3 ... 4.  1.9 5. ]
 [4.2 4.5 3.8 ... 3.1 1.6 4.5]
 [5.  5.  4.4 ... 4.3 2.  4.9]
 ...
 [4.5 4.6 4.9 ... 3.9 2.8 4.7]
 [3.5 3.7 3.  ... 3.9 1.  3.7]
 [4.2 4.4 4.  ... 3.6 1.6 4.4]]


3642